# Ocean Acidification Sea Grant Database
Reyna Ayala

==============================================================

## Table of Contents
[`Installations Required`](#installations-required)  
[`Executable File: advanced_search.py`](#executable-file-advanced_searchpy)  
[`Reference File 1: page.py`](#reference-file-1-pagepy)  
[`Reference File 2: elemnt.py`](#reference-2-elemntpy)

## Installations Required

Firefox  
selenium  
pandas  
openpyxl

## Executable File: advanced_search.py

Import packages

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import openpyxl
import time
import unittest

Import reference files

In [ ]:
import page
#import elemnt

Lists to track all entries and labels

In [ ]:
all_entries = []
all_labels = []

Dictionary, separating entries by label to become columns in Excel spreadsheet

In [ ]:
all_data_dict = {
    "NSGL Document #:" : [],
    "Sea Grant Program/Affiliate:" : [],
    "Title:" : [],
    "Author:" : [],
    "Publication Year :" : [],
    "Publisher:" : [],
    "Publication Type:" : [],
    "Program Report #:" : [],
    "Grant/Contract #:" : [],
    "Project #:" : []
}

Main class: reads website, copies to dictionary, inputs information to Excel spreadsheet

In [ ]:
# Pass unittest as object. Allows us to run unittest.
class Advanced_Search(unittest.TestCase):

    # Establishes Firefox as browser
    def set_up(self):
        self.driver = webdriver.Firefox()
        self.driver.get("http://eos.ucs.uri.edu/EOSWebOPAC/OPAC/Search/AdvancedSearch.aspx")

    # Return to previous page
    def back_one(self):
        self.driver.back()
    
    # Advance to next 100 entries in search
    def next_page(self):
        # presence of element will not be detected until page is fully loaded. following command requires waiting until page is loaded.
        WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.ID, 'ctl00_webopacContentHolder_SearchTitleListControl_titleListNav1_arrowRight')))

        # find arrow element
        arrow = self.driver.find_element(By.ID, 'ctl00_webopacContentHolder_SearchTitleListControl_titleListNav1_arrowRight')
        arrow.click()

    # Search "ocean acidification" in searchbox
    def search_ocean_acidif(self):
        # ensure that page matches main page as defined in page.py
        main_page = page.MainPage(self.driver)
        self.assertTrue(main_page.is_title_matches(), "Advanced Search - OPAC Discovery doesn't match.")

        # enter "ocean acidification" in searchbox
        main_page.search_text("ocean acidification")
        main_page.click_go()

        # establish search results page for future reference as defined in page.py
        self.search_results_page = page.SearchResultsPage(self.driver)
        self.assertTrue(self.search_results_page.is_results_found(), "No results found.")
    
    # Assign elements to keys in dictionary
    def write_to_dict(self, key, value):
        # Author and Project # can have multiple entries for one page, thus these are special cases and must be treated separately
        if "Author:" not in key and "Project #:" not in key:
            # if entry is strictly numbers, change to int for easier data analysis in Excel
            if key in all_data_dict.keys():
                if value.isnumeric():
                    value = int(value)
                all_data_dict[key] += [value]
    
    # Check: do the column categories match labels for the page in question?
    def add_empty(self, labels):
        for k in all_data_dict.keys():
            if k not in labels:
                all_data_dict[k] += ['']

    # Open one result page from main page (of the 247 search results)
    def open_single_result(self, row_num):
        # presence of element will not be detected until page is fully loaded. following command requires waiting until page is loaded.
        WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.ID, 'MainRepeaterDetailRow_'+row_num)))

        # find the search result according to row number
        row_i = self.driver.find_element(By.ID, 'MainRepeaterDetailRow_' + row_num)
        link_i = row_i.find_element(By.CLASS_NAME, 'NoVisitNoUnder')
        link_i.click()
    
    # Collect result for an open page
    def collect_single_result(self, row_num):
        # ensure page is fully loaded
        WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.ID, 'ctl00_ctl00_webopacContentHolder_detailContent_BibliographicDetail_BibDetailRepeater_ctl01_DataCell')))

        # temporary lists
        entry = []
        labels = []

        # ID found from page inspection
        long_id = 'ctl00_ctl00_webopacContentHolder_detailContent_BibliographicDetail_BibDetailRepeater_ctl'
        label = ''
        i = 0
        
        try:
            # stop when label is either 'Notes:' or 'Abstract:'
            while label != 'Notes:' and label != 'Abstract:':
                i += 1
                # if/else necessary as numbers less than ten are single digit in their IDs
                if i < 10:
                    label = self.driver.find_element(By.ID, long_id + '0' + str(i) + '_LabelCell').text
                    labels.append(label)
                    value = self.driver.find_element(By.ID, long_id + '0' + str(i) +'_DataCell').text
                    entry.append(value) #occurring at index i-1
                else:
                    label = self.driver.find_element(By.ID, long_id + str(i) + '_LabelCell').text
                    labels.append(label)
                    value = self.driver.find_element(By.ID, long_id + str(i) +'_DataCell').text
                    entry.append(value)

                self.write_to_dict(label, value)
                
                # keep track of when special cases occur
                if label == 'Author:':
                    author_row = i
                elif label == 'Publication Year :':
                    pubyear_row = i
                elif label == 'Project #:':
                    proj_num_row = i

                # read following label to trigger stop in while loop if necessary
                try:
                    label = self.driver.find_element(By.ID, long_id + '0' + str(i+1) + '_LabelCell').text
                except:
                    label = self.driver.find_element(By.ID, long_id + str(i+1) + '_LabelCell').text
        except:
            pass

        try:
            # author label is always followed by publication year label
            diff = pubyear_row - author_row

            # combine author entries when there exist multiple
            for j in range(1,diff):
                entry[author_row - 1] += '; ' + entry[author_row - 1 + j]
            all_data_dict["Author:"] += [entry[author_row - 1]]
        except:
            pass

        try:
            # project number should be the last labeled category collected. if the length of labels list is greater than the project number row, there are multiple project numbers.
            # iterate through if this is the case
            # if there is no project number, will throw 'UnboundLocalError'
            if proj_num_row != len(labels)-1:
                for j in range(proj_num_row, len(labels)):
                    if labels[j] == '':
                        entry[proj_num_row - 1] += ', ' + entry[j]
            all_data_dict['Project #:'] += [entry[proj_num_row - 1]]
            #print(str(row_num) + ' : ' + all_data_dict['Project #:'][-1])
        except UnboundLocalError:
            #all_data_dict['Project #:'] += ['']
            print('------ ' + str(row_num) + ' labels: ' + str(labels))
            pass
        except IndexError:
            pass

        # If key from dictionary is not one of the labels for the page, add empty entry to the corresponding list
        self.add_empty(labels)

        # Add to global lists
        all_entries.append(entry)
        all_labels.append(labels)

        #print(str(row_num) + ' : ' + all_data_dict['Project #:'][-1])

    # Instructions for using pandas to modify existing Excel spreadsheet
    def write_to_excel(self):
        # Establish dataframe in question
        dataframe = pd.DataFrame(all_data_dict)

        # Select file, set mode to 'a' for append, 'replace' indicates overwrite setting, openpyxl as engine
        writer = pd.ExcelWriter('database.xlsx', mode='a', if_sheet_exists='replace', engine='openpyxl')
        # Formatting
        dataframe.to_excel(writer, sheet_name='Sheet1', startrow=1, index=False, header=False)
        writer.close()

    # Closes browser window
    def tearDown(self):
        self.driver.close()

# Open search result, read values (stored in dict), return to previous page
def loop_through(start_num, end_num, page_num):
    for n in range(start_num, end_num + 1):
        search.open_single_result(str(page_num * 100 + n))
        #print(str(page_num * 100 + n))
        search.collect_single_result(n)
        search.back_one()

# Instantiate an object of class
search = Advanced_Search()

# Call commands
search.set_up()
search.search_ocean_acidif()

# Read through results 1-100, 101-200, 201-247
loop_through(1,100,0)
search.next_page()
loop_through(1,100,1)
search.next_page()
loop_through(1,47,2)

# Insert data in Excel spreadsheet
try:
    search.write_to_excel()
except ValueError:
    print(len(all_data_dict['Project #:']))

# Close browser
search.tearDown()

## Reference File 1: page.py

In [ ]:
from elemnt import BasePageElement
from selenium import webdriver
#from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.common.by import By

class BasePage(object):
    """initialize base page that will be called from all pages"""
    def __init__(self, driver):
        self.driver = driver

class MainPage(BasePage):
    """Home page action methods come here"""

    def is_title_matches(self):
        return "Advanced Search - OPAC Discovery" in self.driver.title
    
    def search_text(self, input_text):
        searchbox = self.driver.find_element(By.CLASS_NAME, 'AdvancedSearchInputBox')
        searchbox.send_keys(input_text)

    def click_go(self):
        searchbox = self.driver.find_element(By.CLASS_NAME, 'AdvancedSearchInputBox')
        searchbox.send_keys(Keys.RETURN)

class SearchResultsPage(BasePage):

    def is_results_found(self):
        return "No results found." not in self.driver.page_source

## Reference File 2: elemnt.py

Note: file not used, but included as example of further developed object-oriented code

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#from selenium.webdriver.common.by import By

class BasePageElement(object):
    """Base page class that is initialized on every page object class."""

    def __set__(self, obj, value):
        self.driver = driver
        try:
            elem = WebDriverWait(self.driver, 30).until(
                EC.presence_of_element_located((By.ID, value))
            )
        finally:
            print("you found me")

    def __get__(self, obj, owner):
        driver = obj.driver
        WebDriverWait(driver, 100).until(
            lambda driver: driver.find_element(self.locator)
        )
        element = driver.find_element(self.locator)
        return element.get_attribute("value")